In [2]:
#import libraries 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime as dt
sns.set_theme()


In [3]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix,roc_curve,roc_auc_score
from sklearn.metrics import accuracy_score,log_loss
from matplotlib import pyplot as plt

C:\Users\derek\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [4]:
df1 = pd.read_csv('df1.csv',index_col=0)
df2 = pd.read_csv('df2.csv',index_col=0)
df3 = pd.read_csv('df3.csv',index_col=0)
df4 = pd.read_csv('df4.csv',index_col=0)
df5 = pd.read_csv('df5.csv',index_col=0)
df6 = pd.read_csv('df6.csv',index_col=0)
df7 = pd.read_csv('df7.csv',index_col=0)
sets=[df1,df2,df3,df4,df5,df6,df7]

In [11]:
y = df1.iloc[:,0]
X = df1.iloc[:,1:]
X=pd.get_dummies(X)
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 466246 entries, 0 to 465539
Data columns (total 95 columns):
 #   Column                                     Non-Null Count   Dtype  
---  ------                                     --------------   -----  
 0   IsWeekend                                  466246 non-null  bool   
 1   y_lon                                      466246 non-null  float64
 2   x_lat                                      466246 non-null  float64
 3   equity_score                               466246 non-null  float64
 4   time_of_day_Afternoon                      466246 non-null  uint8  
 5   time_of_day_Early Morning                  466246 non-null  uint8  
 6   time_of_day_Evening                        466246 non-null  uint8  
 7   time_of_day_Midday                         466246 non-null  uint8  
 8   time_of_day_Morning                        466246 non-null  uint8  
 9   time_of_day_Night                          466246 non-null  uint8  
 10  day_of_w

In [ ]:
%%time
for df in sets:
    y = df.iloc[:,0]
    X = df.iloc[:,1:]
    X=pd.get_dummies(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=1)
    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train_scaled=scaler.transform(X_train)
    X_test_scaled=scaler.transform(X_test)
    clf = RandomForestClassifier(n_estimators=25, random_state = 1,n_jobs=-1)
    model_res = clf.fit(X_train_scaled, y_train)
    y_pred = model_res.predict(X_test_scaled)
    y_pred_prob = model_res.predict_proba(X_test_scaled)
    lr_probs = y_pred_prob[:,1]
    ac = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    cm = confusion_matrix(y_test, y_pred)
    print('Random Forest: Accuracy=%.3f' % (ac))
    print('Random Forest: f1-score=%.3f' % (f1))
    feature_importance = clf.feature_importances_
    # make importances relative to max importance
    feature_importance = 100.0 * (feature_importance / feature_importance.max())[:30]
    sorted_idx = np.argsort(feature_importance)[:30]
    pos = np.arange(sorted_idx.shape[0]) + .5
    sorted_idx.size
    plt.figure(figsize=(10,10))
    plt.barh(pos, feature_importance[sorted_idx], align='center')
    plt.yticks(pos, X.columns[sorted_idx])
    plt.xlabel('Relative Importance')
    plt.title('Variable Importance')
    plt.show()

In [ ]:
%%time
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

for df in sets:
    y = df.iloc[:,0] 
    X = df.iloc[:,1:]
    X=pd.get_dummies(X)
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=1)
#     scaler = preprocessing.StandardScaler().fit(X_train)
#     X_train_scaled=scaler.transform(X_train)
#     X_test_scaled=scaler.transform(X_test)
    param_grid = {'n_neighbors':np.arange(1,50)}
    knn = KNeighborsClassifier()
    knn_cv= GridSearchCV(knn,param_grid,cv=5)
    knn_cv.fit(X,y)
    print("Best Score:" + str(knn_cv.best_score_))
    print("Best Parameters: " + str(knn_cv.best_params_))

In [ ]:
for df in sets:
    y = df.iloc[:,0]
    X = df.iloc[:,1:]
    X=pd.get_dummies(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=1)
    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train_scaled=scaler.transform(X_train)
    X_test_scaled=scaler.transform(X_test)
    xg_cl = xgb.XGBClassifier(objective='reg:squarederror', n_estimators=10, seed=123)
    xg_cl.fit(X_train_scaled,y_train)
    preds = xg_cl.predict(X_test)
    accuracy = float(np.sum(preds==y_test))/y_test.shape[0]
    print("accuracy: %f" % (accuracy))
    feature_importance = xg_cl.feature_importances_
    # make importances relative to max importance
    feature_importance = 100.0 * (feature_importance / feature_importance.max())[:30]
    sorted_idx = np.argsort(feature_importance)[:30]
    pos = np.arange(sorted_idx.shape[0]) + .5
    sorted_idx.size  
    plt.figure(figsize=(10,10))
    plt.barh(pos, feature_importance[sorted_idx], align='center')
    plt.yticks(pos, X.columns[sorted_idx])
    plt.xlabel('Relative Importance')
    plt.title('Variable Importance')
    plt.show()

In [ ]:
%time (print('hello'))

In [ ]:




learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]

for df in sets:
    y = df.iloc[:,0]
    X = df.iloc[:,1:]
    X=pd.get_dummies(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=1)
    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train_scaled=scaler.transform(X_train)
    X_test_scaled=scaler.transform(X_test)
    for learning_rate in learning_rates:
        gb = GradientBoostingClassifier(n_estimators=20, learning_rate = learning_rate, max_features=2, max_depth = 2, random_state = 0)
        gb.fit(X_train_scaled, y_train)
        print("Learning rate: ", learning_rate)
        print("Accuracy score (training): {0:.3f}".format(gb.score(X_train_scaled, y_train)))
        print("Accuracy score (validation): {0:.3f}".format(gb.score(X_test_scaled, y_test)))
        print()

In [ ]:
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier

for df in sets:
    y = df.iloc[:,0]
    X = df.iloc[:,1:]
    X=pd.get_dummies(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=1)
    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train_scaled=scaler.transform(X_train)
    X_test_scaled=scaler.transform(X_test)
    clf = AdaBoostClassifier(n_estimators=10)
    clf.fit(X_train_scaled, y_train)
    clf.predict(y_test.reshape(-1, 1))
    preds = clf.predict(X_test_scaled)
    accuracy = float(np.sum(preds==y_test))/y_test.shape[0]
    print("accuracy: %f" % (accuracy))
    # feature_importance = clf.feature_importances_
    # # make importances relative to max importance
    # feature_importance = 100.0 * (feature_importance / feature_importance.max())[:30]
    # sorted_idx = np.argsort(feature_importance)[:30]
    # pos = np.arange(sorted_idx.shape[0]) + .5
    # sorted_idx.size  
    # plt.figure(figsize=(10,10))
    # plt.barh(pos, feature_importance[sorted_idx], align='center')
    # plt.yticks(pos, X.columns[sorted_idx])
    # plt.xlabel('Relative Importance')
    # plt.title('Variable Importance')
    # plt.show()


In [ ]:
from sklearn import svm
dont run
for df in sets:
    y = df.iloc[:,0] 
    X = df.iloc[:,1:]
    X=pd.get_dummies(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=1)
    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train_scaled=scaler.transform(X_train)
    X_test_scaled=scaler.transform(X_test)
    clf = svm.SVC()
    clf.fit(X_train_scaled, y_train)
    clf.predict(y_test)
    preds = clf.predict(X_test_scaled)
    accuracy = float(np.sum(preds==y_test))/y_test.shape[0]
    print("accuracy: %f" % (accuracy))
    # feature_importance = clf.feature_importances_
    # # make importances relative to max importance
    # feature_importance = 100.0 * (feature_importance / feature_importance.max())[:30]
    # sorted_idx = np.argsort(feature_importance)[:30]
    # pos = np.arange(sorted_idx.shape[0]) + .5
    # sorted_idx.size  
    # plt.figure(figsize=(10,10))
    # plt.barh(pos, feature_importance[sorted_idx], align='center')
    # plt.yticks(pos, X.columns[sorted_idx])
    # plt.xlabel('Relative Importance')
    # plt.title('Variable Importance') 
    # plt.show()